<a href="https://colab.research.google.com/github/anna-marshalova/model-compression-course/blob/main/mbert_compression_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.5 MB/s eta 0:00:00


# Load data

In [ ]:
from datasets import load_dataset

dataset = load_dataset("tyqiangz/multilingual-sentiments", "all")
#dataset = load_dataset("imdb")

Generating train split:   0%|          | 0/270399 [00:00<?, ? examples/s]

----------------------------------------------------------------------------------------------------
                                                text     label  source  \
0    yang memerlukan pemerhatian dan tindakan serius  positive  malaya   
1  sentiasa memikirkan dan merancang inisiatif ba...  positive  malaya   
2  Kita akan tengok daripada pelbagai aspek supay...  positive  malaya   
3  justeru asean perlu mengambil tindakan sebagai...  positive  malaya   
4  @_Niiar_ Jangan punah dulu, aku belum ke labua...   neutral  malaya   

  language  
0    malay  
1    malay  
2    malay  
3    malay  
4    malay  
----------------------------------------------------------------------------------------------------


Generating validation split:   0%|          | 0/10857 [00:00<?, ? examples/s]

----------------------------------------------------------------------------------------------------
                                                text     label  source  \
0  kerja sama apec perlu wujudkan pertumbuhan eko...  positive  malaya   
1    Ada yang disembunyikan, dan itu yang dipercayai  negative  malaya   
2  Apatah lagi penduduk negara ini terdiri daripa...  positive  malaya   
3  tetapi sejak krisis kewangan asia hingga sekar...  negative  malaya   
4  KOTA KINABALU: Warga emas tertua di Sabah mene...  positive  malaya   

  language  
0    malay  
1    malay  
2    malay  
3    malay  
4    malay  
----------------------------------------------------------------------------------------------------


Generating test split:   0%|          | 0/14465 [00:00<?, ? examples/s]

----------------------------------------------------------------------------------------------------
                                                text     label  source  \
0  Sepatutnya berbuat begitu  demi untuk menawark...  positive  malaya   
1  Alhamdulillah, sama2 bantu kerajaan memerangi ...  positive  malaya   
2  Biasanya bantuan disalurkan kepada sekolah ber...  positive  malaya   
3  Kerajaan wajar mengkaji semula had tunggakan c...  positive  malaya   
4  me; those everytime nak beli baju or seluar ta...  positive  malaya   

  language  
0    malay  
1    malay  
2    malay  
3    malay  
4    malay  
----------------------------------------------------------------------------------------------------


In [ ]:
RANDOM_SEED = 42
dataset = dataset.shuffle(seed=RANDOM_SEED)
dataset.set_format(type="torch", columns=["text", "label"])

In [ ]:
NUM_SHARDS = 10
train_dataset = dataset['train'].shard(num_shards=NUM_SHARDS, index=0)
val_dataset = dataset['validation'].shard(num_shards=NUM_SHARDS, index=0)
test_dataset = dataset['test'].shard(num_shards=NUM_SHARDS, index=0)

In [ ]:
id2label = {0: "positive", 1: "neutral", 2:"negative"}
label2id = {"positive" : 0, "neutral": 1, "negative" : 2}
#id2label = {0: "NEGATIVE", 1: "POSITIVE"}
#label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [ ]:
from torch.utils.data import DataLoader

BATCH_SIZE = 32
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Load model

In [ ]:
#MODEL_NAME = 'bert-base-cased'
MODEL_NAME = 'bert-base-multilingual-cased'

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
#model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=3, id2label=id2label, label2id=label2id)
#model = AutoModel.from_pretrained("bert-base-multilingual-cased")

In [ ]:
def tokenize(batch):
        return tokenizer(batch["text"], truncation=True, padding=True, return_tensors="pt")

In [ ]:
from torch.nn import Module, Linear
from transformers import AutoModel


class Model(Module):
    def __init__(self, pretrained_model_name, num_labels, freeze = True, **kwargs):
        super().__init__()
        self.emb = AutoModel.from_pretrained(pretrained_model_name, output_attentions=False, output_hidden_states=False)
        if freeze:
            for param in self.emb.parameters():
                param.requires_grad = False
        hid_size = list(self.emb.parameters())[-1].shape[0]
        self.out = Linear(hid_size, num_labels)

    def forward(self, input_ids, attention_mask, **kwargs):
        x = self.emb(input_ids = input_ids, attention_mask = attention_mask).pooler_output
        return self.out(x)

In [ ]:
#model = Model(pretrained_model_name = MODEL_NAME, num_labels = 3)

In [ ]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
CHECKPOINTS_PATH = '/content/drive/MyDrive/model_compression/model.pt'
model = torch.load(CHECKPOINTS_PATH, map_location = device)

# Code for training

In [ ]:
import gc
def cleanup():
  torch.cuda.empty_cache()
  gc.collect()

In [ ]:
import os
import sys
import torch
from torch.optim import Adam
from torch.nn import CrossEntropyLoss, BCELoss
from torch.nn.functional import softmax
from tqdm.notebook import tqdm


class Trainer:
    def __init__(self, model, device, learning_rate=1e-4, path_to_weights='/content/drive/MyDrive/model_compression/model.pt'):
        self.device = device
        self.model = model.to(self.device)
        self.optimizer = Adam(list(self.model.parameters()), lr=learning_rate)
        self.criterion = CrossEntropyLoss()
        self.path_to_weights = path_to_weights

    def train(self, train_loader, val_loader, num_epochs, save_model=False):
        loss_history = []
        train_history = []
        val_history = []
        for epoch in range(num_epochs):
            print(f'Epoch {epoch + 1}')
            self.model.train()
            loss_accum = 0
            correct = 0
            total = 0
            for batch_num, batch in enumerate(tqdm(train_loader)):
                prediction, y = self.get_prediction(batch)
                loss = self.compute_loss(prediction, y)
                loss_accum += loss
                accuracy, correct, total = self.compute_accuracy(prediction, y, correct, total)
                sys.stdout.write(f'\rLoss: {loss}. Train accuracy: {accuracy}')
                sys.stdout.flush()

            mean_loss = loss_accum / batch_num
            train_accuracy = float(correct) / total
            val_accuracy = self.compute_val_accuracy(val_loader)

            loss_history.append(float(mean_loss))
            train_history.append(train_accuracy)
            val_history.append(val_accuracy)

            print(
               f"\nAverage loss: {mean_loss}, Train accuracy: {train_accuracy}, Val accuracy: {val_accuracy}" )
            if save_model:
                self.save_model()
        val_accuracy = self.compute_val_accuracy(val_loader)
        return loss_history, train_history, val_history

    def get_prediction(self, batch):
        tokenized_batch = tokenize(batch)
        ids = tokenized_batch['input_ids'].to(self.device)
        mask = tokenized_batch['attention_mask'].to(self.device)
        y = batch['label'].to(self.device)
        prediction = softmax(self.model(input_ids=ids, attention_mask=mask), dim=1)
        return prediction, y

    def compute_loss(self, prediction, y):
        loss = self.criterion(prediction, y)
        self.model.zero_grad()
        loss.backward()
        self.optimizer.step()
        return loss.item()

    def compute_accuracy(self, prediction, y_true, correct, total):
        y_pred = torch.argmax(prediction, axis=1)
        correct += torch.sum(y_pred == y_true)
        total += y_true.shape[0]
        accuracy = correct / total
        return accuracy, correct, total

    def compute_val_accuracy(self, val_loader):
        self.model.eval()
        correct = 0
        total = 0
        for batch_num, batch in enumerate(tqdm(val_loader)):
            prediction, y = self.get_prediction(batch)
            accuracy, correct, total = self.compute_accuracy(prediction, y, correct, total)
            sys.stdout.write(f'\rVal accuracy: {accuracy}')
            sys.stdout.flush()
        accuracy = float(correct) / total
        return accuracy

    def save_model(self):
        torch.save(self.model, self.path_to_weights)
        print(f'Model saved to {self.path_to_weights}')

# Training

In [ ]:
trainer = Trainer(model, device, learning_rate = 1e-2, path_to_weights=CHECKPOINTS_PATH)
trainer.train(train_loader, val_loader, num_epochs=10, save_model=True)